## Importing Libraries

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window

**1336. Number of Transactions per Visit (Hard)**

**Table: Visits**

| Column Name   | Type    |
|---------------|---------|
| user_id       | int     |
| visit_date    | date    |

(user_id, visit_date) is the primary key (combination of columns with unique values) for this table.
Each row of this table indicates that user_id has visited the bank in visit_date.
 
**Table: Transactions**

| Column Name      | Type    |
|------------------|---------|
| user_id          | int     |
| transaction_date | date    |
| amount           | int     |

This table may contain duplicates rows.
Each row of this table indicates that user_id has done a transaction of amount in transaction_date.
It is guaranteed that the user has visited the bank in the transaction_date.(i.e The Visits table contains (user_id, transaction_date) in one row)
 
A bank wants to draw a chart of the number of transactions bank visitors did in one visit to the bank and the corresponding number of visitors who have done this number of transaction in one visit.

**Write a solution to find how many users visited the bank and didn't do any transactions, how many visited the bank and did one transaction, and so on.**

The result table will contain two columns:
- transactions_count which is the number of transactions done in one visit.
- visits_count which is the corresponding number of users who did transactions_count in one visit to the bank.

transactions_count should take all values from 0 to max(transactions_count) done by one or more users.

Return the result table ordered by transactions_count.

The result format is in the following example.

**Example 1:**
![](https://fastly.jsdelivr.net/gh/doocs/leetcode@main/solution/1300-1399/1336.Number%20of%20Transactions%20per%20Visit/images/chart.png)

**Input:** 

**Visits table:**

| user_id | visit_date |
|---------|------------|
| 1       | 2020-01-01 |
| 2       | 2020-01-02 |
| 12      | 2020-01-01 |
| 19      | 2020-01-03 |
| 1       | 2020-01-02 |
| 2       | 2020-01-03 |
| 1       | 2020-01-04 |
| 7       | 2020-01-11 |
| 9       | 2020-01-25 |
| 8       | 2020-01-28 |

**Transactions table:**
| user_id | transaction_date | amount |
|---------|------------------|--------|
| 1       | 2020-01-02       | 120    |
| 2       | 2020-01-03       | 22     |
| 7       | 2020-01-11       | 232    |
| 1       | 2020-01-04       | 7      |
| 9       | 2020-01-25       | 33     |
| 9       | 2020-01-25       | 66     |
| 8       | 2020-01-28       | 1      |
| 9       | 2020-01-25       | 99     |

**Output:** 
| transactions_count | visits_count |
|--------------------|--------------|
| 0                  | 4            |
| 1                  | 5            |
| 2                  | 0            |
| 3                  | 1            |

**Explanation:** The chart drawn for this example is shown above.
* For transactions_count = 0, The visits (1, "2020-01-01"), (2, "2020-01-02"), (12, "2020-01-01") and (19, "2020-01-03") did no transactions so visits_count = 4.
* For transactions_count = 1, The visits (2, "2020-01-03"), (7, "2020-01-11"), (8, "2020-01-28"), (1, "2020-01-02") and (1, "2020-01-04") did one transaction so visits_count = 5.
* For transactions_count = 2, No customers visited the bank and did two transactions so visits_count = 0.
* For transactions_count = 3, The visit (9, "2020-01-25") did three transactions so visits_count = 1.
* For transactions_count >= 4, No customers visited the bank and did more than three transactions so we will stop at transactions_count = 3

In [0]:
visits_data_1336 = [
    (1, "2020-01-01"), (2, "2020-01-02"), (12, "2020-01-01"), (19, "2020-01-03"),
    (1, "2020-01-02"), (2, "2020-01-03"), (1, "2020-01-04"), (7, "2020-01-11"),
    (9, "2020-01-25"), (8, "2020-01-28")
]

visits_columns_1336 = ["user_id", "visit_date"]
visits_df_1336 = spark.createDataFrame(visits_data_1336, visits_columns_1336)
visits_df_1336.show()

transactions_data_1336 = [
    (1, "2020-01-02", 120), (2, "2020-01-03", 22), (7, "2020-01-11", 232),
    (1, "2020-01-04", 7), (9, "2020-01-25", 33), (9, "2020-01-25", 66),
    (8, "2020-01-28", 1), (9, "2020-01-25", 99)
]

transactions_columns_1336 = ["user_id", "transaction_date", "amount"]
transactions_df_1336 = spark.createDataFrame(transactions_data_1336, transactions_columns_1336)
transactions_df_1336.show()


+-------+----------+
|user_id|visit_date|
+-------+----------+
|      1|2020-01-01|
|      2|2020-01-02|
|     12|2020-01-01|
|     19|2020-01-03|
|      1|2020-01-02|
|      2|2020-01-03|
|      1|2020-01-04|
|      7|2020-01-11|
|      9|2020-01-25|
|      8|2020-01-28|
+-------+----------+

+-------+----------------+------+
|user_id|transaction_date|amount|
+-------+----------------+------+
|      1|      2020-01-02|   120|
|      2|      2020-01-03|    22|
|      7|      2020-01-11|   232|
|      1|      2020-01-04|     7|
|      9|      2020-01-25|    33|
|      9|      2020-01-25|    66|
|      8|      2020-01-28|     1|
|      9|      2020-01-25|    99|
+-------+----------------+------+



In [0]:
txn_count_df_1336 = transactions_df_1336\
                        .groupBy("user_id", "transaction_date") \
                            .agg(count("*").alias("transactions_count"))

In [0]:
visit_txn_df_1336 = visits_df_1336\
                    .join(txn_count_df_1336,
                        (visits_df_1336.user_id == txn_count_df_1336.user_id) &
                        (visits_df_1336.visit_date == txn_count_df_1336.transaction_date),how="left")\
                            .select(visits_df_1336.user_id,visits_df_1336.visit_date,coalesce(col("transactions_count"), lit(0)).cast("int").alias("transactions_count"))

In [0]:
agg_df_1336 = visit_txn_df_1336\
                .groupBy("transactions_count")\
                    .agg(count("*").alias("visits_count"))

In [0]:
max_count = agg_df_1336.agg(max("transactions_count")).collect()[0][0]
full_range = spark.createDataFrame([(i,) for i in range(max_count + 1)], ["transactions_count"])

In [0]:
full_range\
    .join(agg_df_1336, on="transactions_count", how="left") \
        .fillna(0, subset=["visits_count"]) \
            .orderBy("transactions_count").show()

+------------------+------------+
|transactions_count|visits_count|
+------------------+------------+
|                 0|           4|
|                 1|           5|
|                 2|           0|
|                 3|           1|
+------------------+------------+

